In [ ]:
import os
import torch
import librosa
from torch.utils.data import Dataset, DataLoader
from transformers import Wav2Vec2Model, Wav2Vec2Processor
from tqdm.notebook import tqdm
import numpy as np
import numpy as np
import time  # 시간 측정을 위해 time 라이브러리 추가
# 장치 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 모델 및 프로세서 로드
model_name = "facebook/wav2vec2-large-960h-lv60-self"
processor = Wav2Vec2Processor.from_pretrained(model_name)
model = Wav2Vec2Model.from_pretrained(model_name, torch_dtype=torch.float16).to(device)




def low_pass_filter(audio, sr, cutoff=2000):
    # FFT를 통한 주파수 도메인 변환
    audio_fft = np.fft.rfft(audio)
    frequencies = np.fft.rfftfreq(len(audio), 1/sr)

    # 주파수가 cutoff 이상인 곳은 필터링
    audio_fft[frequencies > cutoff] = 0

    # 역 FFT를 통해 시간 도메인으로 변환
    filtered_audio = np.fft.irfft(audio_fft)
    return filtered_audio

class AudioDataset(Dataset):
    def __init__(self, directory, processor, sr=16000, target_length=16000*2):
        self.directory = directory
        self.processor = processor
        self.sr = sr
        self.target_length = target_length
        self.audio_labels = []
        self.audio_data = []

        folder_list = os.listdir(directory)
        for label in tqdm(folder_list, desc="Processing folders", leave=True):
            label_dir = os.path.join(directory, label)
            if os.path.isdir(label_dir):
                file_list = os.listdir(label_dir)
                for filename in tqdm(file_list, desc=f"Loading files in {label}", leave=False, mininterval=1):
                    if filename.endswith('.mp3') or filename.endswith('.wav'):
                        file_path = os.path.join(label_dir, filename)
                        audio, _ = librosa.load(file_path, sr=sr)
                        filtered_audio = low_pass_filter(audio, sr)  # 필터 적용

                        # 지정된 길이로 오디오를 패딩하거나 자름
                        if len(filtered_audio) < target_length:
                            padding = target_length - len(filtered_audio)
                            filtered_audio = np.pad(filtered_audio, (0, padding), mode='constant')

                        self.audio_data.append(filtered_audio)
                        self.audio_labels.append(label)

        self.label_to_index = {label: idx for idx, label in enumerate(sorted(set(self.audio_labels)))}
        self.indexed_labels = [self.label_to_index[label] for label in self.audio_labels]

    def __len__(self):
        return len(self.audio_data)

    def __getitem__(self, idx):
        try:
            audio = self.audio_data[idx]
            label = self.indexed_labels[idx]
            # 모델 입력 준비
            inputs = self.processor(audio, sampling_rate=self.sr, return_tensors="pt", padding=True).input_values.squeeze(0)
            return inputs, torch.tensor(label)
        except Exception as e:
            print(f"An error occurred at index {idx}: {e}")
            return None  # 대신 오류 발생시 None을 반환하지 않고 기본 값 설정을 고려

# 분류기 정의
class AudioClassifier(torch.nn.Module):
    def __init__(self, feature_dim, num_classes):
        super(AudioClassifier, self).__init__()
        self.fc = torch.nn.Linear(feature_dim, num_classes)

    def forward(self, x):
        x = self.fc(x)
        return x


In [ ]:
import zipfile
import os

# ZIP 파일 경로 설정
zip_file_path = '/content/drive/MyDrive/sound.zip'
unzip_dir = '/content/'

# 압축을 풀 디렉토리 생성
os.makedirs(unzip_dir, exist_ok=True)

# ZIP 파일 열기 및 압축 풀기
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(unzip_dir)

print(f'ZIP 파일이 {unzip_dir}에 성공적으로 풀렸습니다.')



In [ ]:
# 데이터셋 준비 및 데이터 로더 구성
dataset_directory = '/content/sound'  # 데이터셋 폴더 경로
dataset = AudioDataset(dataset_directory, processor)

In [ ]:
# 새 클래스 수에 맞추어 분류기 재정의
num_new_classes = 2  # 새로운 클래스 수
classifier = AudioClassifier(1024, num_new_classes).to(device)

# 손실 함수 및 옵티마이저
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(classifier.parameters(), lr=0.001)


# 학습 루프
def train(model, classifier, processor, device, dataloader, criterion, optimizer, epochs=100):
    model.eval()  # Wav2Vec2 모델은 피처 추출용이므로 eval 모드 유지
    classifier.train()  # 분류기는 학습 모드

    for epoch in range(epochs):
        for batch_idx, (inputs, labels) in enumerate(dataloader):
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()

            with torch.no_grad():
                features = model(inputs.half()).last_hidden_state

            logits = classifier(features.mean(dim=1).float())
            loss = criterion(logits, labels)
            loss.backward()
            optimizer.step()

            if batch_idx % 100 == 0:
                print(f"Epoch: {epoch+1}, Batch: {batch_idx}, Loss: {loss.item()}")

dataloader = DataLoader(dataset, batch_size=200, shuffle=True)
# 학습 시작
train(model, classifier, processor, device, dataloader, criterion, optimizer)

print("Training complete")

# 모델 저장 경로 설정
save_path = '/content/drive/MyDrive/audio_classifier.pth'# 모델의 state_dict를 저장
torch.save(classifier.state_dict(), save_path)